In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import string
import nltk
from nltk.corpus import stopwords
from wordcloud import WordCloud
nltk.download('stopwords')

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

import warnings
warnings.filterwarnings("ignore")




## Load the dataset

In [ ]:
data = pd.read_csv('Dataset/spam_ham_dataset.csv')
data.head()


In [ ]:
data.shape()

In [ ]:
sns.countplot(x='label', data=data)
plt.show()


## Balance the dataset

In [ ]:
ham_msg = data[data['label'] == 'ham']
spam_msg = data[data['label'] == 'spam']

# Downsample spam messages to match the number of ham messages

ham_msg_balanced = ham_msg.sample(n=len(spam_msg), random_state=42)

# Combine the balanced ham messages with the spam messages
balanced_data = pd.concat([ham_msg_balanced,spam_msg]).reset_index(drop=True)

sns.countplot(x='label', data=balanced_data)
plt.title('Balanced Dataset')
plt.show()

## Clean the Text

In [ ]:
balanced_data['text'] = balanced_data['text'].str.replace('Subject: ', '')
# balanced_data['text'] = balanced_data['text'].str.replace('Re: ', '')
# balanced_data['text'] = balanced_data['text'].str.replace('Fwd: ', '')
balanced_data['text'] = balanced_data['text'].str.lower()
balanced_data.head()